# Request and Download OOI Data

This notebook creates requests for data and QARTOD QC test results that are available from OOINet and from the OOI dev1 server. QC tests associated with datasets from OOINet have already been implemented in production by the Data Team. The dev1 server is where datasets with results of QARTOD tests in development are hosted. Access to dev1 is restricted to OOI personnel on the internal network.

The requests built below include the retrieval method, data stream, and either the reference designator or site, node, and sensor combination for a specific instrument to request data through the OOI M2M API. The requested datasets can also be limited to a time period defined by start datetime and end datetime parameters.

After downloading the datasets and performing preprocessing to prepare the data for analysis, the datasets are saved locally to an interim data folder for the next step in testing and analyzing QARTOD test results.

### Create and activate qartod_test environment
From a Git Bash terminal in the qartod_testing working directory, you'll setup the python environment with conda following these steps:

    # configure the OOI python environment
    conda env create -f environment.yml
    conda init # might be required for windows users if environment is not active
    conda activate qartod_test

    # you can check the active environment by running
    conda env list

    # add qartod_testing module to current environment path
    conda develop .

Next, we'll return to the qartod_test environment before adding two other OOI-specific modules from GitHub to this environment. Do this with,

    conda deactivate


### Using OOINet and ooi-data-explorations libraries locally
To use this notebook as-is, users should fork the OOINet and ooi-data-explorations GitHub repos and clone these to their local GitHub directory. The README for ooi-data-explorations has a detailed description for installing the python library as a local development package [here](https://github.com/oceanobservatories/ooi-data-explorations/tree/master/python#obtaining-the-code-and-configuring-the-environment). These instructions will add the path to ooi-data-explorations to the system path in the resulting ooi environment. Since we use both the OOINet and ooi-data-explorations libraries in this notebook, I like to amend the last step to add both libraries to the path of the environment I'm using while working in this directory. 

Instead of:

    # configure the OOI python environment
    conda env create -f environment.yml
    conda init # might be required for windows users if environment is not active
    conda activate ooi

    # you can check the active environment by running
    conda env list

    # install the package as a local development package
    conda develop .

I would do:

    # Check that ooi environment was successfully created
    conda activate ooi
    conda env list

    # install the package as a local development package in ooi environment
    conda develop .
    
    # Exit the ooi environment
    conda deactivate
    conda env list      # no environments should have an "*" next to it at this point

    # Activate qartod_test environment
    conda activate qartod_test
    conda env list

    # install the package as a local development package in qartod_test environment
    conda develop .

Where `.` denotes the current working directory, so users will want to make sure they are still in their local `GitHub/ooi-data-explorations/python/` directory or   `GitHub/OOINet/` directory at this step.

While creating the environment and activating it just to deactivate the environment seems strange, I like having these environments to use later if needed. 

If conda stalls while creating the ooinet environment, that might be caused by libraries in the environment file that are not compatible with your operating system. I start with removing the package build information, and if the problem persists but the error changes slightly then I comment out the problem packages.

### Import modules used in this notebook

In [1]:
# Import libraries available from main conda channels or conda-forge
import xarray
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
import qartod_testing.data_processing as dp

In [3]:
# Import OOINet library
from ooinet import M2M
from ooinet.M2M import AUTH 
from ooinet.Instrument.common import process_file

In [ ]:
# Import functions from ooi-data-explorations library
from ooi_data_explorations import common 
from ooi_data_explorations.common import SESSION

### QARTOD in Production: Request data from the OOINet THREDDS catalog

##### Define data parameters and routines

In [5]:
# Setup parameters needed to request data
refdes = "CP01CNSM-RID27-03-CTDBPC000"              # Coastal Pioneer Array (NES) - Central Surface Mooring CTD Bottom-pumped, is this the same as site, node, sensor?
method = "recovered_inst"                           # non-decimated data from recovered instrument
stream = "ctdbp_cdef_instrument_recovered"          # name of data stream

# Site, node, and sensor info from deconstructed reference designator
[site, node, sensor] = refdes.split('-', 2)

##### Using mostly OOINet module

In [6]:
# Use the gold copy THREDDs datasets

data = dp.ooinet_gold_copy_request(refdes, method, stream)

[########################################] | 100% Completed | 44.95 s


In [7]:
data

<xarray.Dataset>
Dimensions:                                            (time: 3380605)
Coordinates:
  * time                                               (time) datetime64[ns] ...
Data variables: (12/35)
    sea_water_pressure_qc_results                      (time) float32 dask.array<chunksize=(303449,), meta=np.ndarray>
    sea_water_pressure                                 (time) float64 dask.array<chunksize=(303449,), meta=np.ndarray>
    sea_water_electrical_conductivity_qartod_results   (time) float32 dask.array<chunksize=(303449,), meta=np.ndarray>
    sea_water_pressure_qc_executed                     (time) float32 dask.array<chunksize=(303449,), meta=np.ndarray>
    ctd_time                                           (time) datetime64[ns] dask.array<chunksize=(303449,), meta=np.ndarray>
    sea_water_practical_salinity_qc_executed           (time) float32 dask.array<chunksize=(303449,), meta=np.ndarray>
    ...                                                 ...
    sea_water_density                                  (time) float64 dask.array<chunksize=(303449,), meta=np.ndarray>
    depth                                              (time) float64 dask.array<chunksize=(303449,), meta=np.ndarray>
    sea_water_pressure_qartod_results                  (time) float32 dask.array<chunksize=(303449,), meta=np.ndarray>
    sea_water_electrical_conductivity_qartod_executed  (time) |S64 dask.array<chunksize=(303449,), meta=np.ndarray>
    sea_water_temperature                              (time) float64 dask.array<chunksize=(303449,), meta=np.ndarray>
    sea_water_practical_salinity_qartod_executed       (time) |S64 dask.array<chunksize=(303449,), meta=np.ndarray>
Attributes: (12/69)
    node:                               RID27
    comment:                            Data collected from the OOI M2M API a...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_inst
    stream:                             ctdbp_cdef_instrument_recovered
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                40.13678
    lon:                                -70.76978

##### Using ooi_data_explorations modules

In [12]:
# Load data with common module

data = common.load_gc_thredds(site,node,sensor,method,stream,use_dask=True)    # Request the gold copy data through THREDDs catalog

# It looks like the OOINet module method attempts to avoid collecting ancillary files in addition to the requested sensor files which could add time to the download and open dataset step.
# load_gc_thredds() also calls process_file() within gc_collect() so we achieve the same preprocessing as in the preprocess() defined above.

In [9]:
# Make a copy of the data with a unique name

ds_prod = data.copy()
ds_prod

<xarray.Dataset>
Dimensions:                                            (time: 3380605)
Coordinates:
  * time                                               (time) datetime64[ns] ...
Data variables: (12/35)
    sea_water_pressure_qc_results                      (time) float32 dask.array<chunksize=(303449,), meta=np.ndarray>
    sea_water_pressure                                 (time) float64 dask.array<chunksize=(303449,), meta=np.ndarray>
    sea_water_electrical_conductivity_qartod_results   (time) float32 dask.array<chunksize=(303449,), meta=np.ndarray>
    sea_water_pressure_qc_executed                     (time) float32 dask.array<chunksize=(303449,), meta=np.ndarray>
    ctd_time                                           (time) datetime64[ns] dask.array<chunksize=(303449,), meta=np.ndarray>
    sea_water_practical_salinity_qc_executed           (time) float32 dask.array<chunksize=(303449,), meta=np.ndarray>
    ...                                                 ...
    sea_water_density                                  (time) float64 dask.array<chunksize=(303449,), meta=np.ndarray>
    depth                                              (time) float64 dask.array<chunksize=(303449,), meta=np.ndarray>
    sea_water_pressure_qartod_results                  (time) float32 dask.array<chunksize=(303449,), meta=np.ndarray>
    sea_water_electrical_conductivity_qartod_executed  (time) |S64 dask.array<chunksize=(303449,), meta=np.ndarray>
    sea_water_temperature                              (time) float64 dask.array<chunksize=(303449,), meta=np.ndarray>
    sea_water_practical_salinity_qartod_executed       (time) |S64 dask.array<chunksize=(303449,), meta=np.ndarray>
Attributes: (12/69)
    node:                               RID27
    comment:                            Data collected from the OOI M2M API a...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_inst
    stream:                             ctdbp_cdef_instrument_recovered
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                40.13678
    lon:                                -70.76978

### Save datasets for test in production to interim data folder for further processing

In [ ]:
prod_path = dp.build_data_path(refdes, method, stream, 'prod', folder='external') # added this folder='external' as I was updating the notebook. not sure if I want to put requested data in interim or external going forward

ds_prod.to_netcdf(path=prod_path)                           # repeat for ds_prod

### QARTOD in Development: Request data from dev1 server

We may also want to examine new QARTOD tests which are on staging in the Dev-1 environment before they are moved to production. The Development environemt at ooinet-dev1-west.intra.oceanobservatories.org. In order to access data on Dev-1, you need to be granted access and be connected to the CI-West VPN (vpn-west.oceanobservatories.org) at Oregon State.

In [5]:
# Setup parameters needed to request data 
# Check that instrument parameters match an available OOI datasets on dev1 server
# Maybe change this section to look for data sets programatically with ooi-data-explorations functions (list platforms/sites, list methods, list streams,...)

refdes = "CP03ISSM-RID27-03-CTDBPC000"              # Coastal Pioneer Array (NES) - Inshore Surface Mooring Near Surface Instrument Frame - Bottom-pumped CTD
method = "recovered_inst"                           # non-decimated data from recovered instrument
stream = "ctdbp_cdef_instrument_recovered"          # name of data stream

# Site, node, and sensor info from deconstructed reference designator
[site, node, sensor] = refdes.split('-', 2)

# Set optional parameters 
# We specify a date range to control the size of the dataset requested 
params = {
  'beginDT':'2019-09-26T13:50:00.000Z',
  'endDT':'2020-11-01T13:16:00.000Z',
  'format':'application/netcdf',
  'include_provenance':'true',
  'include_annotations':'true'
}

The Dev-1 environment has no "goldcopy" equivalent THREDDs catalog. Instead we'll have to do the normal request and wait for the datasets to be assembled and made available for download.

We are using a different process for downloading data than in the OOINet section since the default URLs that are set within the other functions connect to OOINet. 
The development environment also doesn't have a gold copy, although different functions to request non-gold copy datasets from OOINet exist in the OOINet and ooi-data-explorations modules.


Our choice of URL is similar to the URL used in the M2M example notebook here: https://github.com/ooi-data-review/2018-data-workshops/blob/master/chemistry/examples/quickstart_python.ipynb 
The rest of the data request process through this section is modeled after the linked tutorial above. 

In [6]:
# Connect to ci-west vpn before running this cell
data = dp.dev1_request(site, node, sensor, method, stream, params)

Waiting for Dev 01 to process and prepare data request, this may take up to 20 minutes.
Waiting: 100%|██████████| 400/400 [01:02<00:00,  6.42it/s]


In [7]:
# Make a copy of the data with a unique name

ds_dev = data.copy()
ds_dev

<xarray.Dataset>
Dimensions:                                            (time: 736718)
Coordinates:
  * time                                               (time) datetime64[ns] ...
Data variables: (12/35)
    sea_water_pressure_qc_results                      (time) uint8 dask.array<chunksize=(10,), meta=np.ndarray>
    sea_water_pressure                                 (time) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    sea_water_electrical_conductivity_qartod_results   (time) uint8 dask.array<chunksize=(10,), meta=np.ndarray>
    sea_water_pressure_qc_executed                     (time) uint8 dask.array<chunksize=(10,), meta=np.ndarray>
    ctd_time                                           (time) int32 dask.array<chunksize=(10,), meta=np.ndarray>
    sea_water_practical_salinity_qc_executed           (time) uint8 dask.array<chunksize=(10,), meta=np.ndarray>
    ...                                                 ...
    sea_water_density                                  (time) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    depth                                              (time) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    sea_water_pressure_qartod_results                  (time) uint8 dask.array<chunksize=(10,), meta=np.ndarray>
    sea_water_electrical_conductivity_qartod_executed  (time) object dask.array<chunksize=(10,), meta=np.ndarray>
    sea_water_temperature                              (time) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    sea_water_practical_salinity_qartod_executed       (time) object dask.array<chunksize=(10,), meta=np.ndarray>
Attributes: (12/69)
    node:                               RID27
    comment:                            Data collected from the OOI Dev01 M2M...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_inst
    stream:                             ctdbp_cdef_instrument_recovered
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                40.367077
    lon:                                -70.881735

### Save datasets for test in development to interim data folder for further processing

In [20]:
interim_data = os.path.relpath('../data/interim')           # path to interim data folder from notebook folder

dev_filename = '-'.join(('dev',ds_dev.id,))+'.nc'           # build ds_dev filename from dataset attributes

dev_path=os.path.join(interim_data, dev_filename)           # build full relative path with ds_dev filename

ds_dev.to_netcdf(path=dev_path)                             # provide both relative path and filename for ds_dev in path parameter